In [1]:
# run.py

!pip install transformers
!pip install faiss-cpu
!pip install sentence-transformers
!pip install torch
!pip install gradio

In [ ]:
import sys
sys.path.append('/content/scripts')


In [2]:
from retriever import initialize_retriever, retrieve_top_chunks
from generator import initialize_generator
import logging
from interface import gradio_interface


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Paths to data
faiss_path = "/content/100_chunk_faiss_index.bin"
metadata_path = "/content/100_chunk_metadata.json"

# Retrieval parameters control the selection and filtering of chunks for context retrieval
retriever_params = {
    "initial_top_k": 30,          # Number of top chunks initially considered based on similarity
    "final_top_k": 2,             # Number of final top chunks passed to the generator after filtering
    "similarity_threshold": 1.1  # Threshold for determining if the topic is not covered
}

# Generation parameters control the behavior of the language model in generating responses
generator_params = {
    "max_new_tokens": 120,        # Maximum number of tokens to generate in the response
    "temperature": 0.13,          # Controls randomness in the response; lower values make output more focused
    "repetition_penalty": 1.2,    # Discourages repetition; lower values allow some repeated content
    "top_p": 0.96,                # Top-p sampling (nucleus sampling); keeps only the most likely tokens up to a 0.95 cumulative probability
    "do_sample": True             # Enables sampling for varied responses; set to False for deterministic output
}

In [4]:
def setup():
    """Initialize retrieval and generator components."""
    # Initialize retrieval components
    index, metadata, retriever_model = initialize_retriever(faiss_path, metadata_path)

    # Initialize generator components
    model, tokenizer, streamer = initialize_generator()

    logging.info("Setup complete.")
    return index, metadata, retriever_model, model, tokenizer, streamer


# Run

In [5]:
if __name__ == "__main__":
    # Initialize models and data only once
    index, metadata, retriever_model, model, tokenizer, streamer = setup()


FAISS index loaded successfully.
Metadata loaded successfully.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Retriever model initialized successfully.


In [6]:
# Start the chatbot interface
gradio_interface(index, metadata, retriever_model, model, tokenizer, streamer, retriever_params, generator_params)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://64a8319b4cb329d7c9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
